**This notebook is an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/xgboost).**

---


In this exercise, you will use your new knowledge to train a model with **gradient boosting**.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [1]:
# Set up code checking
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex6 import *
print("Setup Complete")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Setup Complete


You will work with the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course) dataset from the previous exercise. 

![Ames Housing dataset image](https://storage.googleapis.com/kaggle-media/learn/images/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
# 选择基数相对较低的分类列
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]
'''
大多数机器学习模型都不能直接处理分类变量，需要将它们转换为数值形式。常见的转换方法是独热编码（One-Hot Encoding），即为每个唯一的类别创建一个新的二进制特征。如果一个分类变量的基数非常高，那么独热编码会导致数据集的维度大大增加，这可能会导致模型训练时间过长，需要更多的计算资源，甚至可能导致模型过拟合。
选择基数相对较低的分类列是减少分类特征中唯一类别数量的一种方法，有助于避免one-hot编码问题，提高机器学习模型的性能
然而，值得注意的是，选择低基数列并不总是最好的方法，它取决于具体的数据集和当前的问题。在某些情况下，高基数分类特征可能包含可以提高模型准确性的重要信息，可能需要使用更先进的技术来处理它们
选择基数较低的分类列是一个实用的策略，可以帮助我们保持数据集的维度在一个可管理的范围内，同时也可以避免一些与高基数分类变量相关的问题。然而，这也可能会导致一些信息的丢失，因为我们可能会忽略一些可能对目标变量有影响的分类变量。在实际应用中，如何处理高基数分类变量是一个需要权衡的问题，可能需要根据具体的数据和任务来决定。
'''

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

'''
首先，pd.get_dummies()函数用于将分类变量转换为独热编码形式。
例如，如果我们有一个名为"颜色"的列，其中包含"红色"、"蓝色"和"绿色"三个值，
那么pd.get_dummies()会创建三个新的列，分别名为"颜色_红色"、"颜色_蓝色"和"颜色_绿色"。如果原始列中的值是"红色"，那么"颜色_红色"列的值就是1，其他两列的值就是0。

然后，align()函数用于确保训练集和验证集（或测试集）有相同的列。
这是必要的，因为如果某个分类变量在训练集中出现的值在验证集（或测试集）中没有出现，
那么pd.get_dummies()会在训练集和验证集（或测试集）中创建不同的列。
align()函数通过添加缺失的列（并用0填充）和删除额外的列来解决这个问题。
join='left'参数表示我们要保留左边数据集（即训练集）的列，
axis=1表示我们要沿着列的方向（即横向）对齐数据集。
在align()函数中，axis=1表示我们要沿着列的方向对齐数据集，即确保所有数据集有相同的列。如果我们设置axis=0，那么我们将沿着行的方向对齐数据集，即确保所有数据集有相同的行。然而，在大多数情况下，我们更关心的是列的对齐（因为列通常代表特征），所以axis=1用得更多。
'''

'\n首先，pd.get_dummies()函数用于将分类变量转换为独热编码形式。\n例如，如果我们有一个名为"颜色"的列，其中包含"红色"、"蓝色"和"绿色"三个值，\n那么pd.get_dummies()会创建三个新的列，分别名为"颜色_红色"、"颜色_蓝色"和"颜色_绿色"。如果原始列中的值是"红色"，那么"颜色_红色"列的值就是1，其他两列的值就是0。\n\n然后，align()函数用于确保训练集和验证集（或测试集）有相同的列。\n这是必要的，因为如果某个分类变量在训练集中出现的值在验证集（或测试集）中没有出现，\n那么pd.get_dummies()会在训练集和验证集（或测试集）中创建不同的列。\nalign()函数通过添加缺失的列（并用0填充）和删除额外的列来解决这个问题。\njoin=\'left\'参数表示我们要保留左边数据集（即训练集）的列，\naxis=1表示我们要沿着列的方向（即横向）对齐数据集。\n在align()函数中，axis=1表示我们要沿着列的方向对齐数据集，即确保所有数据集有相同的列。如果我们设置axis=0，那么我们将沿着行的方向对齐数据集，即确保所有数据集有相同的行。然而，在大多数情况下，我们更关心的是列的对齐（因为列通常代表特征），所以axis=1用得更多。\n'

# Step 1: Build model

### Part A

In this step, you'll build and train your first model with gradient boosting.

- Begin by setting `my_model_1` to an XGBoost model.  Use the [XGBRegressor](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor) class, and set the random seed to 0 (`random_state=0`).  **Leave all other parameters as default.**
- Then, fit the model to the training data in `X_train` and `y_train`.

# 步骤 1：建立模型
## A 部分
在本步骤中，您将使用梯度提升技术建立并训练第一个模型。

- 首先将 my_model_1 设置为 XGBoost 模型。
- 使用 XGBRegressor 类，并将随机种子设为 0（random_state=0）。其他参数保持默认值。
- 然后，根据 X_train 和 y_train 中的训练数据拟合模型。

In [3]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor(random_state=0)

# Fit the model
my_model_1.fit(X_train, y_train)

# Check your answer
step_1.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#step_1.a.hint()
#step_1.a.solution()

### Part B

Set `predictions_1` to the model's predictions for the validation data.  Recall that the validation features are stored in `X_valid`.

In [5]:

# Get predictions
predictions_1 =  my_model_1.predict(X_valid)

# Check your answer
step_1.b.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#step_1.b.hint()
#step_1.b.solution()

### Part C

Finally, use the `mean_absolute_error()` function to calculate the mean absolute error (MAE) corresponding to the predictions for the validation set.  Recall that the labels for the validation data are stored in `y_valid`.

In [7]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

# Check your answer
step_1.c.check()

Mean Absolute Error: 17662.736729452055


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#step_1.c.hint()
#step_1.c.solution()

# Step 2: Improve the model

Now that you've trained a default model as baseline, it's time to tinker with the parameters, to see if you can get better performance!
- Begin by setting `my_model_2` to an XGBoost model, using the [XGBRegressor](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor) class.  Use what you learned in the previous tutorial to figure out how to change the default parameters (like `n_estimators` and `learning_rate`) to get better results.
- Then, fit the model to the training data in `X_train` and `y_train`.
- Set `predictions_2` to the model's predictions for the validation data.  Recall that the validation features are stored in `X_valid`.
- Finally, use the `mean_absolute_error()` function to calculate the mean absolute error (MAE) corresponding to the predictions on the validation set.  Recall that the labels for the validation data are stored in `y_valid`.

In order for this step to be marked correct, your model in `my_model_2` must attain lower MAE than the model in `my_model_1`. 

# 步骤 2：改进模型
既然已经训练了一个默认模型作为基线，那么现在就该调整一下参数，看看能否获得更好的性能！

- 首先，使用 XGBRegressor 类将 my_model_2 设置为 XGBoost 模型。利用在上一教程中学到的知识，找出如何更改默认参数（如 n_estimators 和 learning_rate）以获得更好的结果。
- 然后，根据 X_train 和 y_train 中的训练数据拟合模型。
    - 将 predictions_2 设置为模型对验证数据的预测。回顾一下，验证特征存储在 X_valid 中。
- 最后，使用 mean_absolute_error() 函数计算与验证集预测相对应的平均绝对误差（MAE）。请注意，验证数据的标签存储在 y_valid 中。
为了使这一步标记为正确，my_model_2 中的模型必须比 my_model_1 中的模型获得更低的 MAE。

In [9]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
my_model_2.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

# Check your answer
step_2.check()

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Mean Absolute Error: 16802.965325342466


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [10]:
# Lines below will give you a hint or solution code
#step_2.hint()
#step_2.solution()

# Step 3: Break the model

In this step, you will create a model that performs worse than the original model in Step 1.  This will help you to develop your intuition for how to set parameters.  You might even find that you accidentally get better performance, which is ultimately a nice problem to have and a valuable learning experience!
- Begin by setting `my_model_3` to an XGBoost model, using the [XGBRegressor](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor) class.  Use what you learned in the previous tutorial to figure out how to change the default parameters (like `n_estimators` and `learning_rate`) to design a model to get high MAE.
- Then, fit the model to the training data in `X_train` and `y_train`.
- Set `predictions_3` to the model's predictions for the validation data.  Recall that the validation features are stored in `X_valid`.
- Finally, use the `mean_absolute_error()` function to calculate the mean absolute error (MAE) corresponding to the predictions on the validation set.  Recall that the labels for the validation data are stored in `y_valid`.

In order for this step to be marked correct, your model in `my_model_3` must attain higher MAE than the model in `my_model_1`. 

# 步骤 3：打破模型
在这一步中，您将创建一个性能比步骤 1 中的原始模型更差的模型。这将有助于你培养如何设置参数的直觉。您甚至可能会发现，您意外地获得了更好的性能，这最终是一个很好的问题，也是一个宝贵的学习经验！

- 首先，使用 XGBRegressor 类将 my_model_3 设置为 XGBoost 模型。利用在上一教程中学到的知识，找出如何更改默认参数（如 n_estimators 和 learning_rate）来设计模型，以获得较高的 MAE。
- 然后，将模型拟合到 X_train 和 y_train 中的训练数据。
- 将 predictions_3 设置为模型对验证数据的预测值。回顾一下，验证特征存储在 X_valid 中。
- 最后，使用 mean_absolute_error() 函数计算与验证集预测相对应的平均绝对误差（MAE）。请注意，验证数据的标签存储在 y_valid 中。
为了使这一步标记为正确，my_model_3 中的模型必须比 my_model_1 中的模型获得更高的 MAE。

In [11]:
# Define the model
my_model_3 = XGBRegressor(n_estimators=10, learning_rate=0.01, n_jobs=4)
# n_estimators=10, learning_rate=0.01 Mean Absolute Error: 164333.40517310574
# n_estimators=10, learning_rate=0.05 mae: 109972.16702563142
# n_estimators=10, learning_rate=0.1 mae: 65859.73539169521
# n_estimators=10, learning_rate=0.5 mae: 20568.527838720034
# n_estimators=10, learning_rate=1  mae: 25948.06064051798

# n_estimators=100, learning_rate=0.01 Mean Absolute Error: 68584.79109589041
# n_estimators=100, learning_rate=0.05, Mean Absolute Error: 17213.82253317637
# n_estimators=100, learning_rate=0.1 Mean Absolute Error: 17388.557055329624
# n_estimators=100, learning_rate=0.5 Mean Absolute Error: 20930.964656464042
# n_estimators=100, learning_rate=1 Mean Absolute Error: 27386.556613869863

# n_estimators=1000, learning_rate=0.01 Mean Absolute Error: 16918.452937714042
# n_estimators=1000, learning_rate=0.05 Mean Absolute Error: 16688.691513270547
# n_estimators=1000, learning_rate=0.1 Mean Absolute Error: 17308.831001177226
# n_estimators=1000, learning_rate=0.5,Mean Absolute Error: 20928.494541952055
# n_estimators=1000, learning_rate=1,Mean Absolute Error: 27386.61764233733

# n_estimators=10000, learning_rate=0.01 Mean Absolute Error: 16836.033872003423
# n_estimators=10000, learning_rate=0.05 Mean Absolute Error: 16681.90836365582
# n_estimators=10000, learning_rate=0.1 Mean Absolute Error: 17308.523932470034
# n_estimators=10000, learning_rate=0.5 Mean Absolute Error: 20928.012949486303
# n_estimators=10000, learning_rate=1 Mean Absolute Error: 27386.61764233733

# Fit the model
my_model_3.fit(X_train, y_train)

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)

# Check your answer
step_3.check()

Mean Absolute Error: 164333.40517310574


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
# Lines below will give you a hint or solution code
#step_3.hint()
#step_3.solution()

In [13]:
# 创建一个字典，其中键是(n_estimators, learning_rate)对，值是对应的MAE
results = {
    (10, 0.01): 164333.40517310574,
    (10, 0.05): 109972.16702563142,
    (10, 0.1): 65859.73539169521,
    (10, 0.5): 20568.527838720034,
    (10, 1): 25948.06064051798,
    (100, 0.01): 68584.79109589041,
    (100, 0.05): 17213.82253317637,
    (100, 0.1): 17388.557055329624,
    (100, 0.5): 20930.964656464042,
    (100, 1): 27386.556613869863,
    (1000, 0.01): 16918.452937714042,
    (1000, 0.05): 16688.691513270547,
    (1000, 0.1): 17308.831001177226,
    (1000, 0.5): 20928.494541952055,
    (1000, 1): 27386.61764233733,
    (10000, 0.01): 16836.033872003423,
    (10000, 0.05): 16681.90836365582,
    (10000, 0.1): 17308.523932470034,
    (10000, 0.5): 20928.012949486303,
    (10000, 1): 27386.61764233733,
}

# 创建一个列表，其中包含所有的n_estimators和learning_rate值
n_estimators_values = sorted(set(key[0] for key in results.keys()))
learning_rate_values = sorted(set(key[1] for key in results.keys()))

# 创建一个二维数组，其中包含每个(n_estimators, learning_rate)对应的MAE
mae_values = np.array([[results.get((n_estimators, learning_rate), np.nan) for learning_rate in learning_rate_values] for n_estimators in n_estimators_values])

# 使用matplotlib创建一个热图
plt.figure(figsize=(10, 8))
plt.imshow(mae_values, cmap='hot', interpolation='nearest')
plt.colorbar(label='MAE')
plt.xticks(np.arange(len(learning_rate_values)), learning_rate_values)
plt.yticks(np.arange(len(n_estimators_values)), n_estimators_values)
plt.xlabel('Learning Rate')
plt.ylabel('Number of Estimators')
plt.title('MAE for different combinations of n_estimators and learning_rate')
plt.show()

# 使用matplotlib创建一个热图，这次我们使用'viridis'颜色映射，它提供了更好的颜色对比度
plt.figure(figsize=(10, 8))
plt.imshow(mae_values, cmap='viridis', interpolation='nearest')
plt.colorbar(label='MAE')
plt.xticks(np.arange(len(learning_rate_values)), learning_rate_values)
plt.yticks(np.arange(len(n_estimators_values)), n_estimators_values)
plt.xlabel('Learning Rate')
plt.ylabel('Number of Estimators')
plt.title('MAE for different combinations of n_estimators and learning_rate')
plt.show()


NameError: name 'np' is not defined

你可以使用Python的GridSearchCV类来自动测试n_estimators和learning_rate的不同组合。这个类会对每个组合进行交叉验证，然后选择最佳的组合。以下是一个可能的实现：

这将打印出最佳的n_estimators和learning_rate值。注意，由于GridSearchCV使用的是负MAE作为评分，所以最佳的参数是使得负MAE最大（即MAE最小）的参数。

In [14]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# 定义要测试的参数
param_grid = {
    'n_estimators': [100*i for i in range(1,10)],
    'learning_rate': [0.01*i for i in range(1,10)],
}

# 创建一个GridSearchCV对象
grid_search = GridSearchCV(XGBRegressor(n_jobs=-1), param_grid, cv=5, scoring='neg_mean_absolute_error')

'''
estimator：这是你想要优化的模型。在你的例子中，这是一个XGBRegressor对象。

param_grid：这是一个字典或者字典的列表，它定义了要搜索的参数的网格。字典的键是参数的名称，值是尝试的参数值的列表。在你的例子中，你正在尝试四个不同的n_estimators值和五个不同的learning_rate值。

scoring：这是一个字符串，定义了模型评估的方法。在你的例子中，你使用的是负的平均绝对误差（MAE）。这是因为在sklearn中，所有的评分函数都是越高越好，所以损失函数（如MAE）的负值被用作评分函数。

cv：这是一个整数，定义了交叉验证的策略。在你的例子中，你使用的是5折交叉验证，这意味着数据集被分成5个部分，模型在4个部分上进行训练，在剩下的部分上进行评估。这个过程重复5次，每次使用不同的部分进行评估。

'''

# 使用训练数据拟合GridSearchCV对象
grid_search.fit(X_train, y_train)

# 打印最佳参数
print(grid_search.best_params_)



KeyboardInterrupt: 

In [ ]:
import pandas as pd

# 从grid对象中提取结果，并将其转换为DataFrame
results = pd.DataFrame(grid_search.cv_results_)

print(results[['param_n_estimators', 'param_learning_rate', 'mean_test_score']])

# 使用matplotlib或seaborn等库来绘制结果。例如，你可以创建一个热图来显示不同参数组合的得分：
import seaborn as sns
import matplotlib.pyplot as plt

pivot = results.pivot('param_n_estimators', 'param_learning_rate', 'mean_test_score')
sns.heatmap(pivot)
plt.show()

# 首先，我们需要将结果数据框中的参数列转换为数值类型，以便在热图中使用
results['param_n_estimators'] = results['param_n_estimators'].astype(int)
results['param_learning_rate'] = results['param_learning_rate'].astype(float)

# 然后，我们创建一个新的数据框，其中的行和列分别对应于不同的参数值，单元格中的值对应于测试得分
pivot = results.pivot('param_n_estimators', 'param_learning_rate', 'mean_test_score')

# 最后，我们使用matplotlib创建热图
plt.figure(figsize=(10, 8))
plt.imshow(pivot, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Mean Test Score')
plt.xticks(np.arange(len(pivot.columns)), pivot.columns)
plt.yticks(np.arange(len(pivot.index)), pivot.index)
plt.xlabel('Learning Rate')
plt.ylabel('Number of Estimators')
plt.title('Mean Test Score for different combinations of n_estimators and learning_rate')
plt.show()


### 模型评估方法改为r2

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# 定义要测试的参数
param_grid = {
    'n_estimators': [100*i for i in range(1,10)],
    'learning_rate': [0.01*i for i in range(1,10)],
}

# 创建一个GridSearchCV对象，将评估指标改为'r2'
grid_search = GridSearchCV(XGBRegressor(n_jobs=-1), param_grid, cv=5, scoring='r2')

# 使用训练数据拟合GridSearchCV对象
grid_search.fit(X_train, y_train)

# 打印最佳参数
print(grid_search.best_params_)

# 从grid对象中提取结果，并将其转换为DataFrame
results = pd.DataFrame(grid_search.cv_results_)

# 首先，我们需要将结果数据框中的参数列转换为数值类型，以便在热图中使用
results['param_n_estimators'] = results['param_n_estimators'].astype(int)
results['param_learning_rate'] = results['param_learning_rate'].astype(float)

# 然后，我们创建一个新的数据框，其中的行和列分别对应于不同的参数值，单元格中的值对应于测试得分
pivot = results.pivot('param_n_estimators', 'param_learning_rate', 'mean_test_score')

# 最后，我们使用matplotlib创建热图
plt.figure(figsize=(10, 8))
plt.imshow(pivot, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Mean Test Score')
plt.xticks(np.arange(len(pivot.columns)), pivot.columns)
plt.yticks(np.arange(len(pivot.index)), pivot.index)
plt.xlabel('Learning Rate')
plt.ylabel('Number of Estimators')
plt.title('Mean Test Score for different combinations of n_estimators and learning_rate')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 提取结果
n_estimators = results['param_n_estimators']
learning_rate = results['param_learning_rate']
scores = results['mean_test_score']

# 创建一个新的图形
fig, ax = plt.subplots()

# 对于每一个学习率，画一条折线图
for lr in learning_rate.unique():
    # 提取这个学习率的结果
    mask = learning_rate == lr
    ax.plot(n_estimators[mask], scores[mask], label=f'Learning Rate: {lr}')

# 添加图例和标签
ax.legend()
ax.set_xlabel('Number of Estimators')
ax.set_ylabel('Mean Test Score')
ax.set_title('Performance of XGBoost with Different Parameters')

# 显示图形
plt.show()


### 改为neg_root_mean_squared_error 

>  比赛的评分规则:
> 
> 提交的评估均方根误差(RMSE)之间的对数的预测值和对数的观察销售价格。(测量原木意味着，预测昂贵房屋和廉价房屋的错误将对结果产生同样的影响。)

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# 定义要测试的参数
param_grid = {
    'n_estimators': [100*i for i in range(1,10)],
    'learning_rate': [0.01*i for i in range(1,10)],
}

# 创建一个GridSearchCV对象，将评估指标改为'r2'
grid_search = GridSearchCV(XGBRegressor(n_jobs=-1), param_grid, cv=5, scoring='neg_root_mean_squared_error')

# 使用训练数据拟合GridSearchCV对象
grid_search.fit(X_train, y_train)

# 打印最佳参数
print(grid_search.best_params_)

# 从grid对象中提取结果，并将其转换为DataFrame
results = pd.DataFrame(grid_search.cv_results_)

# 首先，我们需要将结果数据框中的参数列转换为数值类型，以便在热图中使用
results['param_n_estimators'] = results['param_n_estimators'].astype(int)
results['param_learning_rate'] = results['param_learning_rate'].astype(float)

# 然后，我们创建一个新的数据框，其中的行和列分别对应于不同的参数值，单元格中的值对应于测试得分
pivot = results.pivot('param_n_estimators', 'param_learning_rate', 'mean_test_score')

# 最后，我们使用matplotlib创建热图
plt.figure(figsize=(10, 8))
plt.imshow(pivot, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Mean Test Score')
plt.xticks(np.arange(len(pivot.columns)), pivot.columns)
plt.yticks(np.arange(len(pivot.index)), pivot.index)
plt.xlabel('Learning Rate')
plt.ylabel('Number of Estimators')
plt.title('Mean Test Score for different combinations of n_estimators and learning_rate')
plt.show()


改为随机搜索

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# 定义要搜索的参数
param_distributions = {
    "n_estimators": [10, 100, 1000, 10000],
    "learning_rate": [0.01, 0.05, 0.1, 0.5, 1]
}

# 创建模型
model = XGBRegressor(n_jobs=4)

# 创建RandomizedSearchCV对象
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,  # 运行的随机搜索迭代次数
    scoring='neg_mean_absolute_error',  # 评分标准
    cv=5,  # 交叉验证的折数
    verbose=2,  # 控制详细程度
    random_state=42,  # 随机种子，确保结果的可重复性
    n_jobs=-1,  # 使用所有可用的CPU核心
    error_score='raise'
)

# 运行随机搜索
random_search.fit(X, y)

# 打印最佳参数
print(random_search.best_params_)

import pandas as pd

# 从grid对象中提取结果，并将其转换为DataFrame
results = pd.DataFrame(random_search.cv_results_)

print(results[['param_n_estimators', 'param_learning_rate', 'mean_test_score']])

# 使用matplotlib或seaborn等库来绘制结果。例如，你可以创建一个热图来显示不同参数组合的得分：
import seaborn as sns
import matplotlib.pyplot as plt

pivot = results.pivot('param_n_estimators', 'param_learning_rate', 'mean_test_score')
sns.heatmap(pivot)
plt.show()




# Keep going

Continue to learn about **[data leakage](https://www.kaggle.com/alexisbcook/data-leakage)**.  This is an important issue for a data scientist to understand, and it has the potential to ruin your models in subtle and dangerous ways!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*